In [2]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [3]:
os.environ["TOKEN"] = "AAAAAAAAAAAAAAAAAAAAAI6BhgEAAAAAIQN6sfW6eTYPVsxZo3Eg57bTNkQ%3Dxs4oguCrUEAZZRmxmOH2R9WPDDIDUpgeqEcSzeBkVYzk3w0Nak"

def auth(): 
    return os.getenv("TOKEN")

In [4]:
def create_headers(bearer_token): 
    headers = {"Authorization": "Bearer {}".format("AAAAAAAAAAAAAAAAAAAAAI6BhgEAAAAAIQN6sfW6eTYPVsxZo3Eg57bTNkQ%3Dxs4oguCrUEAZZRmxmOH2R9WPDDIDUpgeqEcSzeBkVYzk3w0Nak")}
    return headers

In [5]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "xbox lang:en"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 15

In [8]:
# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

In [9]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [10]:
import datetime
from dateutil.parser import parse
from dateutil.rrule import rrule, DAILY, MO, TU, WE, TH, FR

start = datetime.datetime(2019,10,28);
end = datetime.datetime(2022,10,27);

daysWithoutWeekend = rrule(
  DAILY,
  byweekday=(MO,TU,WE,TH,FR),
  dtstart=start,
  until=end
)

start_list = []
end_list = []

for entry in daysWithoutWeekend:
    start_list.append(entry.strftime("%Y-%m-%d") + "T00:01:00.000Z")
    end_list.append(entry.strftime("%Y-%m-%d") + "T23:59:00.000Z")

for start, end in zip(start_list, end_list): 
    print(start)
    print(end)
    

2019-10-28T00:01:00.000Z
2019-10-28T23:59:00.000Z
2019-10-29T00:01:00.000Z
2019-10-29T23:59:00.000Z
2019-10-30T00:01:00.000Z
2019-10-30T23:59:00.000Z
2019-10-31T00:01:00.000Z
2019-10-31T23:59:00.000Z
2019-11-01T00:01:00.000Z
2019-11-01T23:59:00.000Z
2019-11-04T00:01:00.000Z
2019-11-04T23:59:00.000Z
2019-11-05T00:01:00.000Z
2019-11-05T23:59:00.000Z
2019-11-06T00:01:00.000Z
2019-11-06T23:59:00.000Z
2019-11-07T00:01:00.000Z
2019-11-07T23:59:00.000Z
2019-11-08T00:01:00.000Z
2019-11-08T23:59:00.000Z
2019-11-11T00:01:00.000Z
2019-11-11T23:59:00.000Z
2019-11-12T00:01:00.000Z
2019-11-12T23:59:00.000Z
2019-11-13T00:01:00.000Z
2019-11-13T23:59:00.000Z
2019-11-14T00:01:00.000Z
2019-11-14T23:59:00.000Z
2019-11-15T00:01:00.000Z
2019-11-15T23:59:00.000Z
2019-11-18T00:01:00.000Z
2019-11-18T23:59:00.000Z
2019-11-19T00:01:00.000Z
2019-11-19T23:59:00.000Z
2019-11-20T00:01:00.000Z
2019-11-20T23:59:00.000Z
2019-11-21T00:01:00.000Z
2019-11-21T23:59:00.000Z
2019-11-22T00:01:00.000Z
2019-11-22T23:59:00.000Z


In [11]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)

                
max_results = 500


def get_tweets(bearer_token, headers, keyword, start_list, end_list, max_results, csv_name):
    
    #Total number of tweets we collected from the loop
    total_tweets = 0
    # Create file
    csvFile = open(csv_name, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
    csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
    csvFile.close()

    for i in range(0,len(start_list)):

        # Inputs
        count = 0 # Counting tweets per time period
        max_count = 100 # Max tweets per time period
        flag = True
        next_token = None

        # Check if flag is true
        while flag:
            # Check if max_count reached
            if count >= max_count:
                break
            print("-------------------")
            print("Token: ", next_token)
            url = create_url(keyword, start_list[i],end_list[i], max_results)
            json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
            result_count = json_response['meta']['result_count']

            if 'next_token' in json_response['meta']:
                # Save the token to use for next call
                next_token = json_response['meta']['next_token']
                print("Next Token: ", next_token)
                if result_count is not None and result_count > 0 and next_token is not None:
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, csv_name)
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)                
            # If no next token exists
            else:
                if result_count is not None and result_count > 0:
                    print("-------------------")
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, csv_name)
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)

                #Since this is the final request, turn flag to false to move to the next time period.
                flag = False
                next_token = None
            time.sleep(5)
    print("Total number of results: ", total_tweets)

In [52]:
get_tweets(bearer_token, headers, "ORCL", start_list, end_list, max_results, "orcl_tweets.csv")

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-10-28T00:01:00.000Z
# of Tweets added from this response:  129
Total # of Tweets added:  129
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-10-29T00:01:00.000Z
# of Tweets added from this response:  117
Total # of Tweets added:  246
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-10-30T00:01:00.000Z
# of Tweets added from this response:  88
Total # of Tweets added:  334
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-10-31T00:01:00.000Z
# of Tweets added from this response:  54
Total # of Tweets added:  388
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-11-01T00:01:00.000Z
# of Tweets added from this respons

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-12-20T00:01:00.000Z
# of Tweets added from this response:  112
Total # of Tweets added:  5285
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-12-23T00:01:00.000Z
# of Tweets added from this response:  87
Total # of Tweets added:  5372
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-12-24T00:01:00.000Z
# of Tweets added from this response:  63
Total # of Tweets added:  5435
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-12-25T00:01:00.000Z
# of Tweets added from this response:  19
Total # of Tweets added:  5454
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2019-12-26T00:01:00.000Z
# of Tweets added from this resp

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-02-13T00:01:00.000Z
# of Tweets added from this response:  130
Total # of Tweets added:  8432
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-02-14T00:01:00.000Z
# of Tweets added from this response:  95
Total # of Tweets added:  8527
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-02-17T00:01:00.000Z
# of Tweets added from this response:  36
Total # of Tweets added:  8563
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-02-18T00:01:00.000Z
# of Tweets added from this response:  83
Total # of Tweets added:  8646
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-02-19T00:01:00.000Z
# of Tweets added from this resp

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-04-08T00:01:00.000Z
# of Tweets added from this response:  33
Total # of Tweets added:  12162
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-04-09T00:01:00.000Z
# of Tweets added from this response:  37
Total # of Tweets added:  12199
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-04-10T00:01:00.000Z
# of Tweets added from this response:  28
Total # of Tweets added:  12227
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-04-13T00:01:00.000Z
# of Tweets added from this response:  33
Total # of Tweets added:  12260
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-04-14T00:01:00.000Z
# of Tweets added from this r

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-06-02T00:01:00.000Z
# of Tweets added from this response:  66
Total # of Tweets added:  13958
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-06-03T00:01:00.000Z
# of Tweets added from this response:  51
Total # of Tweets added:  14009
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-06-04T00:01:00.000Z
# of Tweets added from this response:  61
Total # of Tweets added:  14070
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-06-05T00:01:00.000Z
# of Tweets added from this response:  68
Total # of Tweets added:  14138
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-06-08T00:01:00.000Z
# of Tweets added from this r

...

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqlob362kjg6ovsng91509bsmh6l
Start Date:  2022-03-10T00:01:00.000Z
# of Tweets added from this response:  493
Total # of Tweets added:  80786
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqm31s0zsdb0q85hb8ax2tr4pe9p
Start Date:  2022-03-11T00:01:00.000Z
# of Tweets added from this response:  458
Total # of Tweets added:  81244
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-03-14T00:01:00.000Z
# of Tweets added from this response:  169
Total # of Tweets added:  81413
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-03-15T00:01:00.000Z
# of Tweets added from this response:  260
Total # of Tweets added:  81673
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-----

Endpoint Response Code: 200
-------------------
Start Date:  2022-05-03T00:01:00.000Z
# of Tweets added from this response:  57
Total # of Tweets added:  84895
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-04T00:01:00.000Z
# of Tweets added from this response:  127
Total # of Tweets added:  85022
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-05T00:01:00.000Z
# of Tweets added from this response:  201
Total # of Tweets added:  85223
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-06T00:01:00.000Z
# of Tweets added from this response:  107
Total # of Tweets added:  85330
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-09T00:01:00.000Z
# of Tweets added from this response:  104
Total # of Tweet

Endpoint Response Code: 200
-------------------
Start Date:  2022-06-24T00:01:00.000Z
# of Tweets added from this response:  206
Total # of Tweets added:  90510
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-06-27T00:01:00.000Z
# of Tweets added from this response:  236
Total # of Tweets added:  90746
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-06-28T00:01:00.000Z
# of Tweets added from this response:  103
Total # of Tweets added:  90849
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-06-29T00:01:00.000Z
# of Tweets added from this response:  236
Total # of Tweets added:  91085
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-06-30T00:01:00.000Z
# of Tweets added from this response:  167
Total # of Twee

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-08-18T00:01:00.000Z
# of Tweets added from this response:  63
Total # of Tweets added:  95533
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-08-19T00:01:00.000Z
# of Tweets added from this response:  71
Total # of Tweets added:  95604
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-08-22T00:01:00.000Z
# of Tweets added from this response:  116
Total # of Tweets added:  95720
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-08-23T00:01:00.000Z
# of Tweets added from this response:  108
Total # of Tweets added:  95828
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-08-24T00:01:00.000Z
# of Tweets added from this

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-10-11T00:01:00.000Z
# of Tweets added from this response:  104
Total # of Tweets added:  101898
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-10-12T00:01:00.000Z
# of Tweets added from this response:  101
Total # of Tweets added:  101999
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-10-13T00:01:00.000Z
# of Tweets added from this response:  138
Total # of Tweets added:  102137
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-10-14T00:01:00.000Z
# of Tweets added from this response:  166
Total # of Tweets added:  102303
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-10-17T00:01:00.000Z
# of Tweets added fro

In [43]:
get_tweets(bearer_token, headers, "NFLX", start_list, end_list, max_results, "nflx_tweets.csv")

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqmxpq68fjw6ev2763zkiie9xnct
Start Date:  2022-03-18T00:01:00.000Z
# of Tweets added from this response:  382
Total # of Tweets added:  382
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqncvd085qv1kmdi0ecbamwo3tkt
Start Date:  2022-03-21T00:01:00.000Z
# of Tweets added from this response:  461
Total # of Tweets added:  843
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqnro6wc8v0qp20rrhok2w9isyv1
Start Date:  2022-03-22T00:01:00.000Z
# of Tweets added from this response:  448
Total # of Tweets added:  1291
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqnrwolis41s6eroxs066djcsccd
Start Date:  2022-03-23T00:01:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  1768
--------------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywl833gpi4yuxigjb5tlw96g829p
Start Date:  2022-05-04T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  16269
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywlmtu4gcqrhwyobnmehvdkuxn5p
Start Date:  2022-05-05T00:01:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added:  16763
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywln2cq362w31buccjk54xbvzxx9
Start Date:  2022-05-06T00:01:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  17252
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywm2a33enxgbe1q299j7hkcjz725
Start Date:  2022-05-09T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  17742
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyznxy6loihgz0uwt50fcc65suxdp
Start Date:  2022-06-20T00:01:00.000Z
# of Tweets added from this response:  498
Total # of Tweets added:  32447
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyzocv7u54j1rtxj23r1cyuw3age5
Start Date:  2022-06-21T00:01:00.000Z
# of Tweets added from this response:  484
Total # of Tweets added:  32931
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyzod1nipcf33eifatbzmwr260n0d
Start Date:  2022-06-22T00:01:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  33422
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2kc6ekj1lz61q9zs1cmn7vjh4l9
Start Date:  2022-06-23T00:01:00.000Z
# of Tweets added from this response:  498
Total # of Tweets added:  33920
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5m7yqmxvq4s5lzht5fg2zf5gigt
Start Date:  2022-08-04T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  48683
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5m855f084gx1fm8fs4adb1ox2t9
Start Date:  2022-08-05T00:01:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  49172
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5mncx9xpvapxfbov8m1hiifb14t
Start Date:  2022-08-08T00:01:00.000Z
# of Tweets added from this response:  497
Total # of Tweets added:  49669
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5n25p2kcek5fj12tqwitqk5wuil
Start Date:  2022-08-09T00:01:00.000Z
# of Tweets added from this response:  487
Total # of Tweets added:  50156
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkia0p4pyvdkrwwt54ohg1gr1x9
Start Date:  2022-09-20T00:01:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  64535
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkiiieltg26fstwn5lr65vqscjh
Start Date:  2022-09-21T00:01:00.000Z
# of Tweets added from this response:  438
Total # of Tweets added:  64973
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkxbbopxabjey23ghosimqj11ml
Start Date:  2022-09-22T00:01:00.000Z
# of Tweets added from this response:  482
Total # of Tweets added:  65455
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkxhr2a4ouz9nutuwvv0wjuzd6l
Start Date:  2022-09-23T00:01:00.000Z
# of Tweets added from this response:  456
Total # of Tweets added:  65911
--------

In [56]:
get_tweets(bearer_token, headers, "TWR", start_list, end_list, max_results, "twr_tweets.csv")

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-08-12T00:01:00.000Z
# of Tweets added from this response:  430
Total # of Tweets added:  430
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mdi6wcb4ljpsmsfuz0ea0vfd7y5
Start Date:  2020-08-13T00:01:00.000Z
# of Tweets added from this response:  443
Total # of Tweets added:  873
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mdx1v9bgf79hs1iivl7t0eo5gfx
Start Date:  2020-08-14T00:01:00.000Z
# of Tweets added from this response:  455
Total # of Tweets added:  1328
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mec7glnn1338tzo552ppojtbbel
Start Date:  2020-08-17T00:01:00.000Z
# of Tweets added from this response:  439
Total # of Tweets added:  1767
-------------------
-------------------
Token:  None


-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8sdec28oukm13uqy0g43wfiu83h
Start Date:  2020-09-29T00:01:00.000Z
# of Tweets added from this response:  476
Total # of Tweets added:  15754
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8ss768t4hk898ueuv3g8spnb6v1
Start Date:  2020-09-30T00:01:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  16221
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8ssdkq4jpmt47jvtqx5as6w8ebh
Start Date:  2020-10-01T00:01:00.000Z
# of Tweets added from this response:  476
Total # of Tweets added:  16697
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8ssjyb2qy3fbm3ddziyx151uzjx
Start Date:  2020-10-02T00:01:00.000Z
# of Tweets added from this response:  481
Total # of Tweets added:  17178
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosbv30tlfj8iwptco4qcr8evt3531
Start Date:  2020-11-13T00:01:00.000Z
# of Tweets added from this response:  464
Total # of Tweets added:  30874
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosbvi6fuh5l9gyb0591pdfqapvwql
Start Date:  2020-11-16T00:01:00.000Z
# of Tweets added from this response:  426
Total # of Tweets added:  31300
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-11-17T00:01:00.000Z
# of Tweets added from this response:  427
Total # of Tweets added:  31727
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosbvilcmjy61d4jq09u94kbddpa4d
Start Date:  2020-11-18T00:01:00.000Z
# of Tweets added from this response:  471
Total # of Tweets added:  32198
-------------------
-------------------
Token: 

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foshtdigxtmooa936dxk92xbwrkrjx
Start Date:  2020-12-31T00:01:00.000Z
# of Tweets added from this response:  439
Total # of Tweets added:  46011
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foshtdouitboo7sm67idsj4crxzo8t
Start Date:  2021-01-01T00:01:00.000Z
# of Tweets added from this response:  432
Total # of Tweets added:  46443
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foshtswlhvpp83teipz8hlz45sd7y5
Start Date:  2021-01-04T00:01:00.000Z
# of Tweets added from this response:  443
Total # of Tweets added:  46886
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foshu7nbjvnx2ut354srph6qa06l4t
Start Date:  2021-01-05T00:01:00.000Z
# of Tweets added from this response:  451
Total # of Tweets added:  47337
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosnrnrmvtoe0i3x7ch3d4pbdvivst
Start Date:  2021-02-16T00:01:00.000Z
# of Tweets added from this response:  456
Total # of Tweets added:  60754
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2021-02-17T00:01:00.000Z
# of Tweets added from this response:  442
Total # of Tweets added:  61196
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosns2qun65zu70ybo41gh9u6rfsl9
Start Date:  2021-02-18T00:01:00.000Z
# of Tweets added from this response:  455
Total # of Tweets added:  61651
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosns2zcy9ormdt7tse8gvqwffkkn1
Start Date:  2021-02-19T00:01:00.000Z
# of Tweets added from this response:  471
Total # of Tweets added:  62122
-------------------
-------------------
Token: 

...

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytlv22dip9rbwxdlr4guj72y3af1
Start Date:  2022-04-08T00:01:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added:  199633
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytma5iosn0t963jrv7ptz80rz33x
Start Date:  2022-04-11T00:01:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  200118
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytmag6xa6d357905zfpxwsh6qem5
Start Date:  2022-04-12T00:01:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  200609
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytmp913w4u519as3m9gs6wbugk8t
Start Date:  2022-04-13T00:01:00.000Z
# of Tweets added from this response:  492
Total # of Tweets added:  201101
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywokz7jhqi9v421lunz5eaxfairh
Start Date:  2022-05-25T00:01:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  215765
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywol7pui7lt5b65so4cy20ynlpbx
Start Date:  2022-05-26T00:01:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  216256
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyzkkcgwsl01h6r2oxwd1mvfzmo3h
Start Date:  2022-05-27T00:01:00.000Z
# of Tweets added from this response:  493
Total # of Tweets added:  216749
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyzkzk7vitejwtisof3zmd08gjbel
Start Date:  2022-05-30T00:01:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  217234
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2mvck8lkb1ybjstyp5x1wr9z3wd
Start Date:  2022-07-11T00:01:00.000Z
# of Tweets added from this response:  457
Total # of Tweets added:  231812
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2mvn92dc8wlrsiayccei4sloc59
Start Date:  2022-07-12T00:01:00.000Z
# of Tweets added from this response:  493
Total # of Tweets added:  232305
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2nabtsrm4qednfu8tmsm8u01dvh
Start Date:  2022-07-13T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  232795
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2nai7yyi9v19il55g99vp12cnlp
Start Date:  2022-07-14T00:01:00.000Z
# of Tweets added from this response:  487
Total # of Tweets added:  233282
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz8kqsu6se83lz9csxpyalrjw1dvh
Start Date:  2022-08-25T00:01:00.000Z
# of Tweets added from this response:  496
Total # of Tweets added:  247852
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz8l5jjn7frw4t4ku8e9d518ig399
Start Date:  2022-08-26T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  248342
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz8lkra0khiqt3g0tqbod800nxda5
Start Date:  2022-08-29T00:01:00.000Z
# of Tweets added from this response:  495
Total # of Tweets added:  248837
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz8lkvkyrq76xtiv81kfo3mnzzlod
Start Date:  2022-08-30T00:01:00.000Z
# of Tweets added from this response:  461
Total # of Tweets added:  249298
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbngs66e75ah1oqs9379xgaipmkd
Start Date:  2022-10-11T00:01:00.000Z
# of Tweets added from this response:  498
Total # of Tweets added:  263812
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzejfspkttvkzjonjl03smy1xbhj1
Start Date:  2022-10-12T00:01:00.000Z
# of Tweets added from this response:  496
Total # of Tweets added:  264308
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzejg15sbax0ux7gs8xm556dl49z1
Start Date:  2022-10-13T00:01:00.000Z
# of Tweets added from this response:  483
Total # of Tweets added:  264791
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzejg7mdbulqb6oa6rrv6a6u61w8t
Start Date:  2022-10-14T00:01:00.000Z
# of Tweets added from this response:  493
Total # of Tweets added:  265284
----

In [59]:
get_tweets(bearer_token, headers, "MSFT", start_list, end_list, max_results, "msft_tweets.csv")

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91icjz1rqtl6z8ruj261akwsnzx
Start Date:  2019-10-28T00:01:00.000Z
# of Tweets added from this response:  495
Total # of Tweets added:  495
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91x3abyn2kab5bexywsi5j3r531
Start Date:  2019-10-29T00:01:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added:  989
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91xbsc25kgzmnt7gawpmva4p2m5
Start Date:  2019-10-30T00:01:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  1474
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91xi80jbs2xaf744jsfhcz44kfx
Start Date:  2019-10-31T00:01:00.000Z
# of Tweets added from this response:  496
Total # of Tweets added:  1970
--------------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vgd52041d4xt58fy33ypfcx6ge5
Start Date:  2019-12-12T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  16634
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vgdbhzjqaqgm39am1en8742b5vh
Start Date:  2019-12-13T00:01:00.000Z
# of Tweets added from this response:  497
Total # of Tweets added:  17131
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vgsj8chjwko2sbpjkgv5elq0e4d
Start Date:  2019-12-16T00:01:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  17620
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vh7c0qkgor6b4foo9egyje4jptp
Start Date:  2019-12-17T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  18110
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71engbh3qe22x7x0e2esf90rqjr1
Start Date:  2020-01-28T00:01:00.000Z
# of Tweets added from this response:  475
Total # of Tweets added:  32326
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enots5oe5xtfs7wcy6oqcqes8t
Start Date:  2020-01-29T00:01:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added:  32820
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2hm5wira8j34lcosgnzd7lth9
Start Date:  2020-01-30T00:01:00.000Z
# of Tweets added from this response:  389
Total # of Tweets added:  33209
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2o251ju99tg5ck45qhydgg1vh
Start Date:  2020-01-31T00:01:00.000Z
# of Tweets added from this response:  470
Total # of Tweets added:  33679
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74gygf3j1pzdtx9bx387adrbouwt
Start Date:  2020-03-13T00:01:00.000Z
# of Tweets added from this response:  487
Total # of Tweets added:  47268
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74hdo55z1gfchs8xapnk9is6o2nx
Start Date:  2020-03-16T00:01:00.000Z
# of Tweets added from this response:  492
Total # of Tweets added:  47760
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77dcsvbw60dqa7bkf0i796jdsewt
Start Date:  2020-03-17T00:01:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  48249
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77dczblp38qw3njnmxw9zqim0nzx
Start Date:  2020-03-18T00:01:00.000Z
# of Tweets added from this response:  479
Total # of Tweets added:  48728
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7af8l8vvfosh704ixvtg6vrm53el
Start Date:  2020-04-28T00:01:00.000Z
# of Tweets added from this response:  248
Total # of Tweets added:  61508
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7af8tql9ju3vexmq8l06gw87ot4t
Start Date:  2020-04-29T00:01:00.000Z
# of Tweets added from this response:  444
Total # of Tweets added:  61952
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7af906kj6a59uu0w8s0n9p47btkt
Start Date:  2020-04-30T00:01:00.000Z
# of Tweets added from this response:  484
Total # of Tweets added:  62436
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7afnsycy8cga51tofw855e2kr059
Start Date:  2020-05-01T00:01:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  62921
--------

...

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqmxpskmw0g24bs6pawg04ocpcot
Start Date:  2022-03-18T00:01:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  292626
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqncxj8hond3irmeon1j8xwt3d6l
Start Date:  2022-03-21T00:01:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  293103
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqnrqc7xvp7t7e9y5vi9wc3gdapp
Start Date:  2022-03-22T00:01:00.000Z
# of Tweets added from this response:  447
Total # of Tweets added:  293550
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqnrytxjkyv5pekvgw9o0hkkrr0d
Start Date:  2022-03-23T00:01:00.000Z
# of Tweets added from this response:  471
Total # of Tweets added:  294021
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywl834ns7gm8vd8xy5e32o81oi2l
Start Date:  2022-05-04T00:01:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  308627
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywlmvxn2ayi97ijc4e5tnvmil7r1
Start Date:  2022-05-05T00:01:00.000Z
# of Tweets added from this response:  496
Total # of Tweets added:  309123
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywln2dmmi9py3fet46wwjemtil4t
Start Date:  2022-05-06T00:01:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  309614
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywm2a3zqffgqne62zmlc8qluim4d
Start Date:  2022-05-09T00:01:00.000Z
# of Tweets added from this response:  493
Total # of Tweets added:  310107
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyzny2g1q3j3a3jerx0pyidt9oscd
Start Date:  2022-06-20T00:01:00.000Z
# of Tweets added from this response:  497
Total # of Tweets added:  324945
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyzocv9bx6hlmphsby8dlf9iedful
Start Date:  2022-06-21T00:01:00.000Z
# of Tweets added from this response:  492
Total # of Tweets added:  325437
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyzod3r1ctoig75zj7tfjaybjx4zh
Start Date:  2022-06-22T00:01:00.000Z
# of Tweets added from this response:  495
Total # of Tweets added:  325932
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2kc6fguswb26xttps2c7mfxau0t
Start Date:  2022-06-23T00:01:00.000Z
# of Tweets added from this response:  498
Total # of Tweets added:  326430
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5m7yru6kraz9axerq72574neqv1
Start Date:  2022-08-04T00:01:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  341214
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5m8798hmr4c27wpzm39sx54ag71
Start Date:  2022-08-05T00:01:00.000Z
# of Tweets added from this response:  497
Total # of Tweets added:  341711
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5mnf0ifjiqtjmktgyju3r323mnx
Start Date:  2022-08-08T00:01:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  342200
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5n27t6r5ddq2x0rrg6ngdfl0sql
Start Date:  2022-08-09T00:01:00.000Z
# of Tweets added from this response:  495
Total # of Tweets added:  342695
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkic3x5ukfn96yjia20158dljst
Start Date:  2022-09-20T00:01:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  357454
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkiijwigjq5874tq79aoma1krct
Start Date:  2022-09-21T00:01:00.000Z
# of Tweets added from this response:  458
Total # of Tweets added:  357912
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkxbcl7rpfjztmx3i47ato1kwvx
Start Date:  2022-09-22T00:01:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  358401
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkxjtp5kz342g6ud74vxpe3bz3x
Start Date:  2022-09-23T00:01:00.000Z
# of Tweets added from this response:  478
Total # of Tweets added:  358879
----

In [12]:
get_tweets(bearer_token, headers, "BTC", start_list, end_list, max_results, "btc_tweets.csv")

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91icl5yblfpircgih9nrc3nehrx
Start Date:  2019-10-28T00:01:00.000Z
# of Tweets added from this response:  462
Total # of Tweets added:  462
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91x5djv8fxlww0375y9fuaq3ywt
Start Date:  2019-10-29T00:01:00.000Z
# of Tweets added from this response:  460
Total # of Tweets added:  922
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91xbtj4tth4l35ddsgdto2dfgjh
Start Date:  2019-10-30T00:01:00.000Z
# of Tweets added from this response:  468
Total # of Tweets added:  1390
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnmbxwgjoyw2aguvl38br0d21d9vnh
Start Date:  2019-10-31T00:01:00.000Z
# of Tweets added from this response:  455
Total # of Tweets added:  1845
--------------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vgd52wqfngfoj3obqgnhnwoz059
Start Date:  2019-12-12T00:01:00.000Z
# of Tweets added from this response:  433
Total # of Tweets added:  15554
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vgddkm7km8h0ja7q83sao50i8ot
Start Date:  2019-12-13T00:01:00.000Z
# of Tweets added from this response:  443
Total # of Tweets added:  15997
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vgslazg0d3hqwyaaav4yssdktq5
Start Date:  2019-12-16T00:01:00.000Z
# of Tweets added from this response:  453
Total # of Tweets added:  16450
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6vh7c1xk4r7xxvgsyncr6w5g7ngd
Start Date:  2019-12-17T00:01:00.000Z
# of Tweets added from this response:  453
Total # of Tweets added:  16903
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enieecfe574xv0x8exyhdaia9p
Start Date:  2020-01-28T00:01:00.000Z
# of Tweets added from this response:  466
Total # of Tweets added:  30600
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enoudhg1ztqq3vsq8njy2z5oxp
Start Date:  2020-01-29T00:01:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  31067
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2hmrech6v79scp032r7adbu65
Start Date:  2020-01-30T00:01:00.000Z
# of Tweets added from this response:  457
Total # of Tweets added:  31524
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2o2qpgh57xv5o10ndtqhfqpa5
Start Date:  2020-01-31T00:01:00.000Z
# of Tweets added from this response:  465
Total # of Tweets added:  31989
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74gyihfkjsqn1z98j1k0swl73b7h
Start Date:  2020-03-13T00:01:00.000Z
# of Tweets added from this response:  449
Total # of Tweets added:  45473
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74hdo62gw32j9xx1916hviioxbi5
Start Date:  2020-03-16T00:01:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  45940
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77dcsw8ax9mlnrikm1ackf6k6sn1
Start Date:  2020-03-17T00:01:00.000Z
# of Tweets added from this response:  453
Total # of Tweets added:  46393
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77dd1e89vocklk8a7imjeae3frlp
Start Date:  2020-03-18T00:01:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  46860
--------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7af8tr6svji39fn6koe4n0qkbjb1
Start Date:  2020-04-29T00:01:00.000Z
# of Tweets added from this response:  432
Total # of Tweets added:  60039
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7afnmjksskjd4opgv5x7bf9pjq7x
Start Date:  2020-04-30T00:01:00.000Z
# of Tweets added from this response:  443
Total # of Tweets added:  60482
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7afnszk2esrwrld86gdz4g41qph9
Start Date:  2020-05-01T00:01:00.000Z
# of Tweets added from this response:  426
Total # of Tweets added:  60908
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7ag30px9bpozt2h4u86czwrriirh
Start Date:  2020-05-04T00:01:00.000Z
# of Tweets added from this response:  405
Total # of Tweets added:  61313
--------

...

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqnryvpziutsty5rju6lhkagy2d9
Start Date:  2022-03-23T00:01:00.000Z
# of Tweets added from this response:  423
Total # of Tweets added:  252370
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqo6ro3xxha66dfmf1r6cyplkupp
Start Date:  2022-03-24T00:01:00.000Z
# of Tweets added from this response:  352
Total # of Tweets added:  252722
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqo705ti2eoe15udusnqkxm88fwd
Start Date:  2022-03-25T00:01:00.000Z
# of Tweets added from this response:  383
Total # of Tweets added:  253105
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqom5ugefccmxncachfa6wknafel
Start Date:  2022-03-28T00:01:00.000Z
# of Tweets added from this response:  376
Total # of Tweets added:  253481
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywm2c6x3nvgdkck95nfqqp1tpn5p
Start Date:  2022-05-09T00:01:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  266103
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywmh2xkszfh4kk7xedtq881h1wu5
Start Date:  2022-05-10T00:01:00.000Z
# of Tweets added from this response:  464
Total # of Tweets added:  266567
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywmhbfaboawldvz8qpn5di9esghp
Start Date:  2022-05-11T00:01:00.000Z
# of Tweets added from this response:  470
Total # of Tweets added:  267037
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpywmhjxa925zqndb0pbg9086bi7b1
Start Date:  2022-05-12T00:01:00.000Z
# of Tweets added from this response:  480
Total # of Tweets added:  267517
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2kc8ip1zrwac929tw0w98v3c96l
Start Date:  2022-06-23T00:01:00.000Z
# of Tweets added from this response:  454
Total # of Tweets added:  281037
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2kceyod2pamajob6n17us0dysfx
Start Date:  2022-06-24T00:01:00.000Z
# of Tweets added from this response:  440
Total # of Tweets added:  281477
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2krmp1ii248t7l5tl3z2ltnve65
Start Date:  2022-06-27T00:01:00.000Z
# of Tweets added from this response:  455
Total # of Tweets added:  281932
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz2krv6r2n7jyextcs528js6pqv0d
Start Date:  2022-06-28T00:01:00.000Z
# of Tweets added from this response:  486
Total # of Tweets added:  282418
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5n27ue1ebev72d9fj7p7ig7j799
Start Date:  2022-08-09T00:01:00.000Z
# of Tweets added from this response:  472
Total # of Tweets added:  296261
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5n2gc3k2rcwf6cdi2u1d7rd02gt
Start Date:  2022-08-10T00:01:00.000Z
# of Tweets added from this response:  422
Total # of Tweets added:  296683
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5n2ms2v5wcjaa7xxox9gnpntvnh
Start Date:  2022-08-11T00:01:00.000Z
# of Tweets added from this response:  448
Total # of Tweets added:  297131
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpz5nhfkr6tvphdqzx07cxhf6l6yd9
Start Date:  2022-08-12T00:01:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  297608
----

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbkxjvhli84hus9reb6jk1aw28ot
Start Date:  2022-09-23T00:01:00.000Z
# of Tweets added from this response:  481
Total # of Tweets added:  311568
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzblcrlkf5cnmzhwwosxx7ey3rz3x
Start Date:  2022-09-26T00:01:00.000Z
# of Tweets added from this response:  483
Total # of Tweets added:  312051
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzbld03kb3da2eeqvua7jcpygf5od
Start Date:  2022-09-27T00:01:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  312540
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzblrqu80fs44uixhbwjrhdzx8ebh
Start Date:  2022-09-28T00:01:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  313007
----

In [ ]:
get_tweets(bearer_token, headers, "ZLDSF", start_list, end_list, max_results, "zldsf_tweets.csv")

In [ ]:
get_tweets(bearer_token, headers, "MMC", start_list, end_list, max_results, "mmc_tweets.csv")